# Thành viên nhóm:
## Nguyễn Hoàng Bảo       22110284
## Trần Thị Kim Chung     22110288
## Ngô Trung Hiếu         22110323

# Demo Bayes

## 1. Import thư viện

In [9]:
import pandas as pd
import numpy as np
from collections import defaultdict

## 2. Tải dữ liệu từ file csv

In [16]:
df_buy_computer = pd.read_csv("../data/Buys_Computer.csv")
df_buy_computer.head()

,age,income,student,credit_rating,buys_computer
0,<=30,high,no,fair,no
1,<=30,high,no,excelent,no
2,31…40,high,no,fair,yes
3,>40,medium,no,fair,yes
4,>40,low,yes,fair,yes


## 3. Các hàm cần thiết

### 3.1 Hàm tính xác suất với Laplace Correction

- Tham số đầu vào:
    - df (pandas.DataFrame): Tập dữ liệu đầu vào dạng DataFrame, chứa các cột thuộc tính (features) và cột mục tiêu (target).
    - target_col (str, mặc định='buys_computer'): Tên cột trong DataFrame đại diện cho lớp mục tiêu (ví dụ: buys_computer với giá trị 'yes' hoặc 'no').
    - alpha (float, mặc định=1): Hệ số Laplace Correction, được thêm vào để tránh xác suất bằng 0. Giá trị mặc định là 1 (Laplace smoothing).

- Hàm trả về một tuple gồm 3 phần tử:
    - prior_probs (dict): Từ điển lưu trữ xác suất tiên nghiệm của từng lớp mục tiêu (P(buys_computer)).
    - conditional_probs (defaultdict): Từ điển lồng nhau lưu trữ xác suất điều kiện P(feature_value | buys_computer) cho từng giá trị của từng thuộc tính, ứng với từng lớp.
    - classes (numpy.ndarray): Mảng chứa các giá trị duy nhất của lớp mục tiêu (ví dụ: ['yes', 'no']).

In [11]:
def calculate_probabilities(df, target_col='buys_computer', alpha=1):
    # Lớp mục tiêu
    classes = df[target_col].unique()
    class_counts = df[target_col].value_counts()
    total_samples = len(df)
    
    # Lưu trữ xác suất
    prior_probs = {}
    conditional_probs = defaultdict(lambda: defaultdict(dict))
    
    # Tính xác suất tiên nghiệm P(buys_computer)
    for c in classes:
        prior_probs[c] = (class_counts[c] + alpha) / (total_samples + len(classes) * alpha)
    
    # Tính xác suất điều kiện P(feature|buys_computer)
    features = [col for col in df.columns if col != target_col]
    for feature in features:
        feature_values = df[feature].unique()
        for c in classes:
            class_df = df[df[target_col] == c]
            for val in feature_values:
                count = len(class_df[class_df[feature] == val])
                # Laplace Correction: (count + alpha) / (total + alpha * number of values)
                conditional_probs[feature][val][c] = (count + alpha) / (len(class_df) + len(feature_values) * alpha)
    
    return prior_probs, conditional_probs, classes

## 3. Hàm dự đoán

- Tham số đầu vào:
    - instance (dict): Một từ điển chứa các cặp key-value, trong đó key là tên thuộc tính (feature) và value là giá trị của thuộc tính cho mẫu dữ liệu mới. Ví dụ: {'age': '<=30', 'income': 'medium', 'student': 'yes', 'credit_rating': 'excellent'}.
    - prior_probs (dict): Từ điển chứa xác suất tiên nghiệm của từng lớp, ví dụ: {'yes': 0.4, 'no': 0.6}.
    - conditional_probs (defaultdict): Từ điển lồng nhau chứa xác suất điều kiện P(feature_value | class) cho từng giá trị của từng thuộc tính, ứng với từng lớp. Cấu trúc: conditional_probs[feature][value][class].
    - classes (numpy.ndarray hoặc list): Danh sách các giá trị duy nhất của lớp mục tiêu, ví dụ: ['yes', 'no'].
    - features (list): Danh sách các thuộc tính (features) được sử dụng trong mô hình, ví dụ: ['age', 'income', 'student', 'credit_rating'].

In [12]:
def predict(instance, prior_probs, conditional_probs, classes, features):
    max_prob = -1
    predicted_class = None
    
    for c in classes:
        prob = prior_probs[c]
        for feature, value in instance.items():
            prob *= conditional_probs[feature][value][c]
        
        if prob > max_prob:
            max_prob = prob
            predicted_class = c
    
    return predicted_class


## 4. Thực thi chương trình dự đoán

- Tính xác xuất

In [13]:
prior_probs, conditional_probs, classes = calculate_probabilities(df_buy_computer)

print(f"Xác suất mua máy tính (buy_computer): {prior_probs}")

# Chuyển defaultdict thành dict
conditional_probs_dict = dict(conditional_probs)

# In từng thuộc tính
for feature in conditional_probs_dict:
    print(f"\nXác suất điều kiện cho thuộc tính '{feature}':")
    for value in conditional_probs_dict[feature]:
        print(f"  Giá trị '{value}':")
        for cls in classes:
            print(f"    P({value} | {cls}) = {conditional_probs_dict[feature][value][cls]:.4f}")
            
print(f"Tên classes: {classes}")

Xác suất mua máy tính (buy_computer): {'no': np.float64(0.375), 'yes': np.float64(0.625)}

Xác suất điều kiện cho thuộc tính 'age':
  Giá trị '<=30':
    P(<=30 | no) = 0.4444
    P(<=30 | yes) = 0.2308
  Giá trị '31…40':
    P(31…40 | no) = 0.1111
    P(31…40 | yes) = 0.3077
  Giá trị '>40':
    P(>40 | no) = 0.3333
    P(>40 | yes) = 0.3077
  Giá trị '31...40':
    P(31...40 | no) = 0.1111
    P(31...40 | yes) = 0.1538

Xác suất điều kiện cho thuộc tính 'income':
  Giá trị 'high':
    P(high | no) = 0.3750
    P(high | yes) = 0.2500
  Giá trị 'medium':
    P(medium | no) = 0.3750
    P(medium | yes) = 0.4167
  Giá trị 'low':
    P(low | no) = 0.2500
    P(low | yes) = 0.3333

Xác suất điều kiện cho thuộc tính 'student':
  Giá trị 'no':
    P(no | no) = 0.7143
    P(no | yes) = 0.3636
  Giá trị 'yes':
    P(yes | no) = 0.2857
    P(yes | yes) = 0.6364

Xác suất điều kiện cho thuộc tính 'credit_rating':
  Giá trị 'fair':
    P(fair | no) = 0.3750
    P(fair | yes) = 0.5833
  Giá trị 'e

- Dữ liệu cần dự đoán

In [ ]:
new_instance = {
    'age': '<=30',
    'in come': 'medium',
    'student': 'yes',
    'credit_rating': 'excellent'
}
new_instance

{'age': '<=30',
 'in come': 'medium',
 'student': 'yes',
 'credit_rating': 'excellent'}

- Dự đoán

In [15]:
features = ['age', 'income', 'student', 'credit_rating']
prediction = predict(new_instance, prior_probs, conditional_probs, classes, features)

KeyError: 'no'

- In kết quả

In [ ]:
print(f"Dữ liệu dự đoán: {new_instance}")
print(f"Kết quả dự đoán buys_computer: {prediction}")

Dữ liệu dự đoán: {'age': '<=30', 'in come': 'medium', 'student': 'yes', 'credit_rating': 'excellent'}
Kết quả dự đoán buys_computer: yes
